In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings(action='once')
import matplotlib.pyplot as plt
from functools import reduce
import math
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

C:\Users\sagar\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\sagar\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [3]:
os.chdir('D:\\Purdue\\Spring - mod 1\\IndustryPracticum\\Data')

In [4]:
sales_BC = pd.read_csv('D:\\Purdue\\Spring - mod 1\\IndustryPracticum\\Data\\SalesData_-BC&MB_fixed.csv')
sales_SK = pd.read_csv('D:\\Purdue\\Spring - mod 1\\IndustryPracticum\\Data\\SalesData_-SK_fixed.csv')
sales_AB = pd.read_csv('D:\\Purdue\\Spring - mod 1\\IndustryPracticum\\Data\\SalesData_AB_fixed.csv')

C:\Users\sagar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
sales = pd.concat([sales_BC,sales_SK])
sales = pd.concat([sales,sales_AB])

In [6]:
sales = sales.rename(columns = {'Crop Year': 'crop_yr','Product Brand': 'Prod_Brand','Product Category': 'Prod_Category'})

In [7]:
phi_data = sales[['PHI_area_cd','PHI_district_cd','PHI_territory_cd']]

In [8]:
columns = ['PHI_area_cd', 'PHI_district_cd','PHI_territory_cd']
sales.drop(columns, inplace=True, axis=1)

In [10]:
sales.head(2)

,Authorization,Grower_AIN,city_name,province_name,Dealer_AIN,Dealer_City_Name,Dealer_Prov,crop_yr,Season,Business,Prod_Category,Prod_Brand,Rolled product name,SumOfSumOfConverted Quantity,SumOfSumOfConverted Acres,SumOfSumOfConverted Total_MSRP
0,Authorized,10556,Cambridge,ON,CA14R-0005108,Westbourne,MB,2016,Spring,Brevant,Canola Seed,Canola,Nexera Canola Hybrid 1012 RR HV 22.7Kg Bag,33.0,346.5,22164.45
1,Authorized,10556,Cambridge,ON,CA14R-0005108,Westbourne,MB,2017,Spring,Brevant,Canola Seed,Canola,Nexera Canola 1012 RR VI 22.7Kg,6.0,63.0,4290.00


In [16]:
sales.province_name.unique()

array(['ON', 'MB', 'NB', 'SK', 'BC', 'AB', 'NU', 'QC', 'UNKNOWN', 'YT',
       'NL', 'NS', 'PE', 'NT'], dtype=object)

# price based aggregation

In [89]:
agg1 = sales.groupby(['Grower_AIN','Business'])['SumOfSumOfConverted Total_MSRP'].sum().unstack().fillna(0)       

In [90]:
agg1.head()

Business,Brevant,Crop Protection,Pioneer,Specialties
Grower_AIN,,,,
1000020,0.0,0.00,0.0,358.9
1000028,0.0,0.00,0.0,691.4
1000029,0.0,7419.20,0.0,0.0
1000030,0.0,134.80,0.0,0.0
1000050,0.0,28519.73,0.0,0.0


In [ ]:
#basket = (df[df['Dealer_Prov'] =='SK']
 #         .groupby(['Grower_AIN', 'Prod_Category'])['SumOfSumOfConverted Acres']
  #        .sum().unstack().reset_index().fillna(0)
   #       .set_index('Grower_AIN'))

In [91]:
agg1['total']= agg1['Brevant'] + agg1['Crop Protection'] + agg1['Pioneer'] + agg1['Specialties']

pandas.core.series.Series

In [92]:
grower_location = sales[['Grower_AIN','city_name','province_name']]

In [93]:
grower_location.head()

,Grower_AIN,city_name,province_name
0,10556,Cambridge,ON
1,10556,Cambridge,ON
2,10556,Cambridge,ON
3,10556,Cambridge,ON
4,10556,Cambridge,ON


In [94]:
grower_location=grower_location.drop_duplicates(subset='Grower_AIN', keep="last")

In [98]:
grower_location = agg1.merge(grower_location, on = 'Grower_AIN')

In [103]:
grower_location.to_csv('grower_data.csv')

In [104]:
grower_location.head()

,Grower_AIN,Brevant,Crop Protection,Pioneer,Specialties,total,city_name,province_name
0,1000020,0.0,0.00,0.0,358.9,358.90,Drayton Valley,AB
1,1000028,0.0,0.00,0.0,691.4,691.40,Oliver,BC
2,1000029,0.0,7419.20,0.0,0.0,7419.20,Richmound,SK
3,1000030,0.0,134.80,0.0,0.0,134.80,Richmond,BC
4,1000050,0.0,28519.73,0.0,0.0,28519.73,Wadena,SK
